# Dependencies

For ZnTrack there are two different ways to set up dependencies:

1. Node/Stage based dependencies
2. File based dependencies

## Node dependencies
We will first look at Node based dependencies starting from a RandomNumber `Hello World` example.
In our first stage we create a random number and then we add another Node that depends on this one.
We can do this very easily by setting the `dvc.deps` in the following way
```py
dependency: Stage = dvc.deps(Stage(load=True))
```

This allows us to access all properties of the `dependency` attribute.

In [ ]:
!git init
!dvc init

In [2]:
from zntrack import Node, dvc, config
from random import randrange
from pathlib import Path

In [3]:
config.nb_name = "dependencies.ipynb"

In [4]:
@Node()
class RandomNumber:
    maximum = dvc.params()
    number = dvc.result()
    
    def __call__(self, maximum):
        self.maximum = maximum
        
    def run(self):
        self.number = float(randrange(self.maximum))

@Node()
class ComputePower:
    random_number: RandomNumber = dvc.deps(RandomNumber(load=True))
    number = dvc.result()
    power = dvc.params()
    
    def __call__(self, power = 2):
        self.power = power
    
    def run(self):
        self.number = self.random_number.number ** self.power

2021-10-26 15:01:39,675 (WARNING): Jupyter support is an experimental feature! Please save your notebook before running this command!
Submit issues to https://github.com/zincware/ZnTrack.


[NbConvertApp] Converting notebook dependencies.ipynb to script


2021-10-26 15:01:43,156 (WARNING): Jupyter support is an experimental feature! Please save your notebook before running this command!
Submit issues to https://github.com/zincware/ZnTrack.


[NbConvertApp] Writing 3150 bytes to dependencies.py
[NbConvertApp] Converting notebook dependencies.ipynb to script
[NbConvertApp] Writing 3150 bytes to dependencies.py


We can now create the stages the usual way and look at the outcomes

In [5]:
RandomNumber()(maximum=16)
ComputePower()(power=2.)

2021-10-26 15:01:46,607 (WARNING): --- Writing new DVC file! ---
2021-10-26 15:01:46,608 (WARNING): Overwriting existing configuration!
2021-10-26 15:01:49,984 (INFO): Creating 'dvc.yaml'
Adding stage 'RandomNumber' in 'dvc.yaml'

To track the changes with git, run:

	git add outs/.gitignore dvc.yaml

2021-10-26 15:01:49,990 (WARNING): --- Writing new DVC file! ---
2021-10-26 15:01:49,990 (WARNING): Overwriting existing configuration!
2021-10-26 15:01:53,061 (INFO): Adding stage 'ComputePower' in 'dvc.yaml'

To track the changes with git, run:

	git add outs/.gitignore dvc.yaml



In [6]:
!dvc repro

Running stage 'RandomNumber':                                         core>
> python3 -c "from src.RandomNumber import RandomNumber; RandomNumber(load=True).run()"
Generating lock file 'dvc.lock'                                                 
Updating lock file 'dvc.lock'

Running stage 'ComputePower':
> python3 -c "from src.ComputePower import ComputePower; ComputePower(load=True).run()"
Updating lock file 'dvc.lock'                                                   

To track the changes with git, run:

	git add dvc.lock
Use `dvc push` to send your updates to remote storage.


In [7]:
print(f'{RandomNumber(load=True).number} ^ {ComputePower(load=True).power} = {ComputePower(load=True).number}')

2021-10-26 15:01:58,971 (WARNING): Jupyter support is an experimental feature! Please save your notebook before running this command!
Submit issues to https://github.com/zincware/ZnTrack.


[NbConvertApp] Converting notebook dependencies.ipynb to script


2021-10-26 15:02:02,543 (WARNING): Jupyter support is an experimental feature! Please save your notebook before running this command!
Submit issues to https://github.com/zincware/ZnTrack.


[NbConvertApp] Writing 3150 bytes to dependencies.py
[NbConvertApp] Converting notebook dependencies.ipynb to script


2021-10-26 15:02:05,880 (WARNING): Jupyter support is an experimental feature! Please save your notebook before running this command!
Submit issues to https://github.com/zincware/ZnTrack.


...

[NbConvertApp] Writing 3150 bytes to dependencies.py
[NbConvertApp] Converting notebook dependencies.ipynb to script


2021-10-26 15:02:13,022 (WARNING): Jupyter support is an experimental feature! Please save your notebook before running this command!
Submit issues to https://github.com/zincware/ZnTrack.


[NbConvertApp] Writing 3150 bytes to dependencies.py
[NbConvertApp] Converting notebook dependencies.ipynb to script


5.0 ^ 2.0 = 25.0


[NbConvertApp] Writing 3150 bytes to dependencies.py


## File dependencies
The second approach is depending on files.
We do this by creating a file from our random number first.
We then use the path to that file as our dependency.
In the end we compare both methods and also show the graph that was built for us by DVC.

We can do set the file dependency very easily in the same way by passing `pathlib.Path` or `str` to the `dvc.deps`.
As every  `dvc.<...>` it also supports lists.
```py
dependency: Path = dvc.deps([Path('some_file.txt'), 'some_other_file.txt'])
```

In [8]:
@Node()
class WriteToFile:
    random_number: RandomNumber = dvc.deps(RandomNumber(load=True))
    file: Path = dvc.outs(Path('random_number.txt'))
    
    def run(self):
        self.file.write_text(str(self.random_number.number))
        
@Node()
class PowerFromFile:
    file: Path = dvc.deps(WriteToFile(load=True).file)
    number = dvc.result()
    power = dvc.params(2)
    
    def run(self):
        number = float(self.file.read_text())
        self.number = number ** self.power
        
@Node()
class ComparePowers:
    power_deps = dvc.deps([PowerFromFile(load=True), ComputePower(load=True)])
    
    def run(self):
        assert self.power_deps[0].number == self.power_deps[1].number

2021-10-26 15:02:16,535 (WARNING): Jupyter support is an experimental feature! Please save your notebook before running this command!
Submit issues to https://github.com/zincware/ZnTrack.


[NbConvertApp] Converting notebook dependencies.ipynb to script


2021-10-26 15:02:20,092 (WARNING): Jupyter support is an experimental feature! Please save your notebook before running this command!
Submit issues to https://github.com/zincware/ZnTrack.


[NbConvertApp] Writing 3150 bytes to dependencies.py
[NbConvertApp] Converting notebook dependencies.ipynb to script


2021-10-26 15:02:23,622 (WARNING): Jupyter support is an experimental feature! Please save your notebook before running this command!
Submit issues to https://github.com/zincware/ZnTrack.


[NbConvertApp] Writing 3150 bytes to dependencies.py
[NbConvertApp] Converting notebook dependencies.ipynb to script
[NbConvertApp] Writing 3150 bytes to dependencies.py


In [9]:
WriteToFile()()
PowerFromFile()()
ComparePowers()()

2021-10-26 15:02:26,983 (WARNING): --- Writing new DVC file! ---
2021-10-26 15:02:26,983 (WARNING): Overwriting existing configuration!
2021-10-26 15:02:30,032 (INFO): Adding stage 'WriteToFile' in 'dvc.yaml'

To track the changes with git, run:

	git add dvc.yaml outs/.gitignore .gitignore

2021-10-26 15:02:30,039 (WARNING): --- Writing new DVC file! ---
2021-10-26 15:02:30,039 (WARNING): Overwriting existing configuration!
2021-10-26 15:02:33,165 (INFO): Adding stage 'PowerFromFile' in 'dvc.yaml'

To track the changes with git, run:

	git add dvc.yaml outs/.gitignore

2021-10-26 15:02:33,175 (WARNING): --- Writing new DVC file! ---
2021-10-26 15:02:33,176 (WARNING): Overwriting existing configuration!
2021-10-26 15:02:36,409 (INFO): Adding stage 'ComparePowers' in 'dvc.yaml'

To track the changes with git, run:

	git add dvc.yaml outs/.gitignore



In [10]:
!dvc dag

                +--------------+                  
                | RandomNumber |                  
                +--------------+                  
                **             ***                
             ***                  ***             
           **                        **           
 +-------------+                       **         
 | WriteToFile |                        *         
 +-------------+                        *         
        *                               *         
        *                               *         
        *                               *         
+---------------+               +--------------+  
| PowerFromFile |               | ComputePower |  
+---------------+               +--------------+  
                **             ***                
                  ***        **                   
                     **    **                     
                +---------------+                 
                | ComparePowers

In [11]:
!dvc repro

Stage 'RandomNumber' didn't change, skipping                          core>
Stage 'ComputePower' didn't change, skipping
Running stage 'WriteToFile':
> python3 -c "from src.WriteToFile import WriteToFile; WriteToFile(load=True).run()"
Updating lock file 'dvc.lock'                                                   

Running stage 'PowerFromFile':
> python3 -c "from src.PowerFromFile import PowerFromFile; PowerFromFile(load=True).run()"
Updating lock file 'dvc.lock'                                                   

Running stage 'ComparePowers':
> python3 -c "from src.ComparePowers import ComparePowers; ComparePowers(load=True).run()"
Updating lock file 'dvc.lock'                                                   

To track the changes with git, run:

	git add dvc.lock
Use `dvc push` to send your updates to remote storage.


If we now look at our `dvc.yaml` we can see that for our Node dependencies we rely on the `outs/<stage_name>.json` while for the file dependency it is directly connect to the passed file.

In [12]:
from IPython.display import Pretty, display

display(Pretty("dvc.yaml"))

stages:
  RandomNumber:
    cmd: python3 -c "from src.RandomNumber import RandomNumber; RandomNumber(load=True).run()"
    deps:
    - src/RandomNumber.py
    params:
    - config/zntrack.json:
      - RandomNumber.params
    outs:
    - outs/RandomNumber.json
  ComputePower:
    cmd: python3 -c "from src.ComputePower import ComputePower; ComputePower(load=True).run()"
    deps:
    - outs/RandomNumber.json
    - src/ComputePower.py
    params:
    - config/zntrack.json:
      - ComputePower.params
    outs:
    - outs/ComputePower.json
  WriteToFile:
    cmd: python3 -c "from src.WriteToFile import WriteToFile; WriteToFile(load=True).run()"
    deps:
    - outs/RandomNumber.json
    - src/WriteToFile.py
    outs:
    - outs/WriteToFile.json
    - random_number.txt
  PowerFromFile:
    cmd: python3 -c "from src.PowerFromFile import PowerFromFile; PowerFromFile(load=True).run()"
    deps:
    - random_number.txt
    - src/PowerFromFile.py
    params:
    - config/zntrack.json:
      - P